In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr, zscore
from tqdm import tqdm
import sys

sys.path.append('../code/scripts/useful_modules')

# Some useful functions for plotting that I wrote previously
from plot_functions import *
from regression_functions import *

In [2]:
all_RPKM = pd.read_csv(
    '../code/ReadLengthMapExperimentResults/tables/AllRNASeq.Normalized.RPKM.bed.gz', sep='\t',
                   index_col=3)

CDS_len = all_RPKM.Length
gene_len = all_RPKM.end - all_RPKM.start

In [3]:

intron_annot = pd.read_csv('../data/IntronAnnotationsFromYang.tsv.gz', sep='\t')
intron_annot['junction'] = intron_annot[['chrom', 'start', 'end', 'strand']].astype(str).agg(':'.join, axis=1)
long_table = pd.read_csv(
    '/project2/yangili1/bjf79/ChromatinSplicingQTLs/code/SplicingAnalysis/CombinedJuncTables/YRI.tsv.gz', sep='\t'
)
#long_table = pd.read_csv("../code/ReadLengthMapExperimentResults/JuncCountsTidy/LongTable.txt.gz", sep='\t')
long_table = long_table.loc[long_table.RepNumber == '1.junc'].drop_duplicates()
long_table = long_table.loc[~((long_table.IndID == 'NA18855') & (long_table.Dataset == 'chRNA.Expression.Splicing'))]#.assay.unique()

In [6]:
long_table = long_table.loc[long_table.Dataset == 'chRNA.Expression.Splicing']

In [7]:
# Adjustment needed to match the introns in Yang's annotation...
long_table.stop += 1

long_table = long_table.merge(intron_annot, left_on=['chrom', 'start', 'stop', 'strand'], 
                 right_on=['chrom', 'start', 'end', 'strand'])


# long_table = long_table.merge(intron_annot, left_on=['chrom', 'start', 'end', 'strand', 'gene', 'NewAnnotation'], 
#                  right_on=['chrom', 'start', 'end', 'strand', 'gene', 'NewAnnotation'])



/project2/mstephens/cfbuenabadn/anaconda3/envs/jnotebook/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [10]:
pc_counts = long_table.loc[long_table.SuperAnnotation.isin(['AnnotatedJunc_ProductiveCodingGene',
                                                            'UnannotatedJunc_ProductiveCodingGene'])
                          ].groupby(['gene', 'IndID']).Count.sum().reset_index()

In [11]:
pc_counts

,gene,IndID,Count
0,ENSG00000000419.13,NA18486,1167
1,ENSG00000000419.13,NA18497,1745
2,ENSG00000000419.13,NA18498,2245
3,ENSG00000000419.13,NA18499,1595
4,ENSG00000000419.13,NA18502,1874
...,...,...,...
1081148,ENSG00000288684.1,NA19238,11
1081149,ENSG00000288684.1,NA19239,11
1081150,ENSG00000288684.1,NA19247,4
1081151,ENSG00000288684.1,NA19257,4
